<h1>THIS NOTEBOOK ANALYZES THE FACTS ABOUT DATA SCIENTISTS & ML ENGINEERS USING THE KAGGLE SURVEY DATA 2020</h1>
    
<h3><ol>
    <li><a href="#time">Outlier Detection in Duration of Survey</a></li>
    <li><a href="#dist">Frequency Distribution of Roles</a></li>
    <li><a href="#age">Age Groups of Data Scientist & ML Engineer</a></li>
    <li><a href="#edu">Education of Data Scientist & ML Engineer</a></li>
    <li><a href="#gen">Gender of Data Scientist & ML Engineer</a></li>
    <li><a href="#cexp">Coding Experience of Data Scientist & ML Engineer</a></li>
    <li><a href="#npl">Number of Programming Languages Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#pl">Programming Languages Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#mpl">Multiple Programming Languages Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#nide">Number of IDEs Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#ide">IDEs Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#mide">Multiple IDEs Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#nhnb">Number of Hosted Notebook Products Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#hnb">Hosted Notebook Products Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#mhnb">Multiple Hosted Notebook Products Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#cpds">Computing Platforms Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#aspir">Recommended Language to Learn First By Data Science Aspirants</a></li>
    <li><a href="#shw">Specialized Harware Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#ndvl">Number of Data Visualization Libraries Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#dvlu">Data Visualization Libraries Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#muldvl">Multiple Data Visualization Libraries Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#mlexp">Years of ML Experience of Data Scientist & ML Engineer</a></li>
    <li><a href="#nmlfw">Number of Machine Learning Frameworks Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#mlfwu">Machine Learning Frameworks Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#mulmlfw">Multiple Machine Learning Frameworks Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#nmlalg">Number of Machine Learning Algorithms Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#mlalg">Machine Learning Algorithms Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#mulmlalg">Multiple Machine Learning Algorithms Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#ncvalg">Number of Computer Vision Methods Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#cvalg">Computer Vision Methods Used By Data Scientist & ML Engineer</a></li>
    <li><a href="#mulcvalg">Multiple Computer Vision Methods Used By Data Scientist & ML Engineer</a></li>
    </ol>
    </h3>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv("../input/kaggle-survey-2020/kaggle_survey_2020_responses.csv",header=1)
data.shape


data.loc[(data['In which country do you currently reside?'] == 'United Kingdom of Great Britain and Northern Ireland'),'In which country do you currently reside?'] = "UK"
data.loc[(data['In which country do you currently reside?'] == 'United States of America'),'In which country do you currently reside?'] = "USA"
data.loc[(data['In which country do you currently reside?'] == 'Iran, Islamic Republic of...'),'In which country do you currently reside?'] = "Iran"
data.loc[(data['In which country do you currently reside?'] == 'Republic of Korea'),'In which country do you currently reside?'] = "South Korea"
data.loc[(data['In which country do you currently reside?'] == 'United Arab Emirates'),'In which country do you currently reside?'] = "UAE"

data["id"] = range(len(data))
data["time_in_mins"] = np.round(data['Duration (in seconds)'] / 60,2)
data["time_in_hrs"] = np.round(data['time_in_mins'] / 60,2)

<a id="time"></a><h1>Outlier Detection in Duration of Survey</h1>

In [ ]:
def outlier_detect(x):
    q1 = x.quantile(0.25)
    q3 = x.quantile(0.75)
    iqr = q3 - q1
    min_ = q1 - (1.5 * iqr)
    max_ = q3 + (1.5 * iqr)
    return min_,max_


time_wo = data.loc[(data["Duration (in seconds)"]<outlier_detect(data["Duration (in seconds)"])[1]) & \
                    (data["Duration (in seconds)"]>outlier_detect(data["Duration (in seconds)"])[0]),"Duration (in seconds)"].copy()

time_outliers = data.loc[data["Duration (in seconds)"].isin(time_wo)==False,"Duration (in seconds)"].copy()

fig = plt.figure(figsize=(15,7))
plt.scatter(x=(time_wo.index),y=time_wo,color="palegreen",marker="x")
plt.scatter(x=(time_outliers.index),y=time_outliers,color="tomato",marker="x")
plt.legend(["Non-outliers","Outliers"])
plt.title("Outliers in Duration of Survey Using IQR Method")
plt.xlabel("Duration (in seconds)")
plt.ylabel("Frequency (log scale)")

for i in [0,0.01,0.03,0.05,0.1,0.15,0.25,0.5,0.75,0.85,0.9,0.95,0.99,1]:
    plt.axhline(y=np.round(data['Duration (in seconds)'].quantile(i),2),color="blue",linestyle="--")
    plt.text(x=-980,y=np.round(data['Duration (in seconds)'].quantile(i)*np.log(2.9),2),s=f"p{int(i*100)}: {np.round(data['Duration (in seconds)'].quantile(i)/60,2)} mins",fontsize=9)
plt.yscale("log",base=2)

<h3>The outliers in duration are computed using IQR method. However,the method couldn't capture the outliers at the lower end. There are records with duration less than a minute, which are not marked as outlier by the IQR method. But, indeed those records might be illegitimate considering the pace at which the survey was responded to.</h3>

In [ ]:
sns.catplot(x='What is your age (# years)?',y='Duration (in seconds)',data=data,aspect=3,height=6,kind="boxen",order=data['What is your age (# years)?'].sort_values().unique());
plt.yscale("log",base=2)

<h3>The median duration is moderately rising with age. But it started to flatten post 45-49 age band. But it has reduced at the 70+ age band.

In [ ]:
def univariate_dist(col,sort_index,title,width,height,xlab,ylab,txt_adjust_x,txt_adjust_y,logx,logy,txt_rot,txt_font):
    data_dist = data[col].value_counts().copy()
    if sort_index == True:
        data_dist = data_dist.sort_index().copy()
    
    data_dist.plot.bar(figsize=(width,height),title=title,color="black")
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    if logx == True:
        plt.xscale("log")
    if logy == True:
        plt.yscale("log")
    for i,s in enumerate(data_dist.index):
        if logy == False:
            plt.text(x=i-txt_adjust_x,y=data_dist[i]+txt_adjust_y,s=f'{data_dist[s]: ,}')
        else:
            plt.text(x=i-txt_adjust_x,y=data_dist[i]*np.log(txt_adjust_y),s=f'{data_dist[s]: ,}',rotation=txt_rot,fontsize=txt_font)
            

            
            
def bivariate_dist(fill_txt,fill_txt2,excl,idx,col,norm,bar_width,width,height,title,xlab,ylab,legend_loc,legend_font,legend_ncol,txt_font,txt_lim,legend_alpha):
    data_dist = pd.crosstab(index=data[idx],columns=data[col])
    
    role_cnt = data[idx].value_counts()
    data_dist=data_dist.merge(role_cnt,how="left",left_index=True,right_index=True).copy()
    for i in data_dist.columns:
        data_dist.loc[:,i] = data_dist.loc[:,i] / data_dist.iloc[:,-1]
    
    data_dist.drop(columns=idx,inplace=True)
    
    
    data_dist1 = data_dist.copy()
    data_dist["sum_total"] = data_dist.sum(axis=1)
    
    for i in data_dist.columns:
        data_dist.loc[:,i] = data_dist.loc[:,i] / data_dist.iloc[:,-1]
    
    data_dist.drop(columns="sum_total",inplace=True)
    
    
    if col == 'What is your age (# years)?':
        data_dist = data_dist.loc[:,['18-21','22-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-69','70+']].copy()
    if col == 'For how many years have you been writing code and/or programming?':
        data_dist = data_dist.loc[:,['I have never written code','< 1 years','1-2 years','3-5 years','5-10 years','10-20 years','20+ years']].copy()
    if col == 'What is the highest level of formal education that you have attained or plan to attain within the next 2 years?':
        data_dist = data_dist.loc[:,['No formal education past high school','Some college/university study without earning a bachelor’s degree',
                                    'Bachelor’s degree','Professional degree','Master’s degree','Doctoral degree','I prefer not to answer']].copy()
    if col == 'Approximately how many times have you used a TPU (tensor processing unit)?':
        data_dist = data_dist.loc[:,['Never','Once',
                                    '2-5 times','6-25 times','More than 25 times']].copy()
    
    
    
    data_dist.plot(kind="barh",stacked=True,width=bar_width,figsize=(width,height),title=title,colormap="tab20");
    plt.legend(loc=legend_loc,fontsize=legend_font,ncol=legend_ncol,framealpha=legend_alpha)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.xticks([])

    for n,i in enumerate(data_dist.index):
        l = np.cumsum(data_dist.loc[i,:])
        l = pd.Series([0]).append(l[:-1])
        l = np.array(l).copy()
        for k,j in enumerate(data_dist.columns):
            if np.round(data_dist1.loc[i,j]*100,1)>txt_lim:
                plt.text(x=l[k],y=n-0.1,s=str(np.round(data_dist1.loc[i,j]*100,1))+"%",fontsize=txt_font)
    facts = ""
    for indx in list(data_dist1.index):
        dat_row = data_dist1.loc[indx,:].sort_values(ascending=False).copy()
        if (indx == "Data Scientist") | (indx == "Machine Learning Engineer"):
            for colum in list(dat_row.index):
                if colum not in excl:
                    
                    facts+= "'" + " ".join((str(np.round(dat_row.loc[colum]*100,2)) + "%' of Respondants Who is/was a " + str(indx) + str(fill_txt) + "'" + str(colum)+"'" + str(fill_txt2)).split()) +"\n"
            facts+="\n\n"

    plt.figtext(x=0.1,y=0.1,s="*Proportions may not add up to 100% due to the presence of missing values")
    plt.show()
    print(facts)

    
    
    
    
def bivariate_dist_2(fill_txt,fill_txt2,excl,match_str,idx,new_idx,new_col,bar_width,width,height,xlab,ylab,title,legend_loc,legend_font,legend_ncol,txt_adjust_y,txt_lim,font,legend_alpha):
    cols = pd.Series(data.columns)[pd.Series(data.columns).str.contains(match_str,regex=False)].copy()
    df = pd.DataFrame()
    for i in list(cols):
        df1 = data.loc[:,[idx,i]].copy()
        df1.dropna(inplace=True)
        df1.columns = [new_idx,new_col]
        df = df.append(df1).copy()






    data_dist = pd.crosstab(index=df[new_idx],
                            columns=df[new_col])
    
    role_cnt = data[idx].value_counts()
    data_dist=data_dist.merge(role_cnt,how="left",left_index=True,right_index=True).copy()
    for i in data_dist.columns:
        data_dist.loc[:,i] = data_dist.loc[:,i] / data_dist.iloc[:,-1]
    
    data_dist.drop(columns=idx,inplace=True)
    
    
    

    
    
    data_dist1 = data_dist.copy()
    data_dist["sum_total"] = data_dist.sum(axis=1)
    
    for i in data_dist.columns:
        data_dist.loc[:,i] = data_dist.loc[:,i] / data_dist.iloc[:,-1]
    
    data_dist.drop(columns="sum_total",inplace=True)
    
    


    data_dist.plot(kind="barh",stacked=True,width=bar_width,figsize=(width,height),title=title,colormap="tab20");
    plt.legend(loc=legend_loc,fontsize=legend_font,ncol=legend_ncol,framealpha=legend_alpha)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.xticks([])
    #plt.yscale("log")
    #plt.xscale("log")
    
    

    for n,i in enumerate(data_dist.index):
        l = np.cumsum(data_dist.loc[i,:])
        l = pd.Series([0]).append(l[:-1])
        l = np.array(l).copy()
        for k,j in enumerate(data_dist.columns):
            if np.round(data_dist1.loc[i,j]*100,1)>txt_lim:
                plt.text(x=l[k],y=n-txt_adjust_y,s=str(np.round(data_dist1.loc[i,j]*100,1))+"%",fontsize=font)
    facts = ""
    for indx in list(data_dist1.index):
        dat_row = data_dist1.loc[indx,:].sort_values(ascending=False).copy()
        if (indx == "Data Scientist") | (indx == "Machine Learning Engineer"):
            for colum in list(dat_row.index):
                if colum not in excl: 
                    facts+= "'" + " ".join((str(np.round(dat_row.loc[colum]*100,2)) + "%' of Respondants Who is/was a " + str(indx) + str(fill_txt) + "'" + str(colum)+"'" + str(fill_txt2)).split()) +"\n"
            facts+="\n\n"
    plt.figtext(x=0.1,y=0.1,s="*Proportions will not add up to 100%, since the question has multiple responses.")
    plt.show()
    print(facts)

    
    
    

def bivariate_dist_5(fill_txt,fill_txt2,excl,match_str,lang,idx,new_idx,new_col,bar_width,width,height,xlab,ylab,title,legend_loc,legend_font,legend_ncol,txt_adjust_y,txt_lim,font,legend_alpha):
    
    
    cols = pd.Series(data.columns)[pd.Series(data.columns).str.contains(match_str,regex=False)].copy()
    
    
    cols1 = list()
    for i in lang:
        if any(cols.str.contains("- Selected Choice - "+i,regex=False)):
            cols1.append(cols[cols.str.contains("- Selected Choice - "+i,regex=False)].reset_index(drop=True)[0])
    
  
    
    dff = data.loc[:,cols1].copy()
    dff.fillna("",inplace=True)
    dff[new_col] = dff.apply(lambda x: ";".join(pd.Series(x).str.strip().sort_values()),axis=1)
    while any(dff[new_col].str.contains(";;",regex=False)):
        dff[new_col] = dff[new_col].str.replace(";;",";",regex=False)
    dff[new_col] = dff[new_col].str.replace(";$"," ",regex=True)
    dff[new_col] = dff[new_col].str.replace("^;"," ",regex=True)
    dff[new_col] = dff[new_col].apply(lambda x: " ".join(x.strip().split()))
    dff = dff.loc[dff[new_col]!="",:].copy()
    dff = pd.concat([data[idx],dff[new_col]],axis=1).copy()
    dff.columns = [new_idx,new_col]
    
    df = data.loc[:,cols].copy()
    df.fillna("",inplace=True)
    df[new_col] = df.apply(lambda x: ";".join(pd.Series(x).str.strip().sort_values()),axis=1)
    while any(df[new_col].str.contains(";;",regex=False)):
        df[new_col] = df[new_col].str.replace(";;",";",regex=False)
    df[new_col] = df[new_col].str.replace(";$"," ",regex=True)
    df[new_col] = df[new_col].str.replace("^;"," ",regex=True)
    df[new_col] = df[new_col].apply(lambda x: " ".join(x.strip().split()))
    df = df.loc[(df[new_col]!="") & (df[new_col]!=" ") & (df[new_col]!=";") & (df[new_col]!="; ") & (df[new_col]!=" ;"),:].copy()
    df = pd.concat([data[idx],df[new_col]],axis=1).copy()
    df.columns = [new_idx,new_col]
    


    
    data_dist = pd.crosstab(index=df[new_idx],
                            columns=df[new_col])
    
    role_cnt = data[idx].value_counts()
    data_dist=data_dist.merge(role_cnt,how="left",left_index=True,right_index=True).copy()
    for i in data_dist.columns:
        data_dist.loc[:,i] = data_dist.loc[:,i] / data_dist.iloc[:,-1]

    data_dist.drop(columns=idx,inplace=True)
    
    data_dist = data_dist.loc[:,list(pd.Series(data_dist.columns)[pd.Series(data_dist.columns).isin(dff[new_col].unique())])].copy()
    
    data_dist1 = data_dist.copy()
    data_dist["sum_total"] = data_dist.sum(axis=1)
    
    for i in data_dist.columns:
        data_dist.loc[:,i] = data_dist.loc[:,i] / data_dist.iloc[:,-1]
    
    data_dist.drop(columns="sum_total",inplace=True)
    
   

    
    data_dist.plot(kind="barh",stacked=True,width=bar_width,figsize=(width,height),title=title,colormap="tab20");
    plt.legend(loc=legend_loc,fontsize=legend_font,ncol=legend_ncol,framealpha=legend_alpha)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.xticks([])
    

    for n,i in enumerate(data_dist.index):
        l = np.cumsum(data_dist.loc[i,:])
        l = pd.Series([0]).append(l[:-1])
        l = np.array(l).copy()
        for k,j in enumerate(data_dist.columns):
            if np.round(data_dist1.loc[i,j]*100,1)>txt_lim:
                plt.text(x=l[k],y=n-txt_adjust_y,s=str(np.round(data_dist1.loc[i,j]*100,1))+"%",fontsize=font)
    facts = ""
    for indx in list(data_dist1.index):
        dat_row = data_dist1.loc[indx,:].sort_values(ascending=False).copy()
        if (indx == "Data Scientist") | (indx == "Machine Learning Engineer"):
            for colum in list(dat_row.index):
                if colum not in excl:
                    
                    facts+= "'" + " ".join((str(np.round(dat_row.loc[colum]*100,2)) + "%' of Respondants Who is/was a " + str(indx) + str(fill_txt) + "'" + str(colum)+"'" + str(fill_txt2)).split()) +"\n"
            facts+="\n\n"
    plt.figtext(x=0.1,y=0.1,s="*Proportions will not add up to 100%, since combinations of only a subset of responses are visualized.")
    plt.show()
    print(facts)
    

    
    
    
    
def bivariate_dist_3(fill_txt,fill_txt2,excl,match_str,count_col,idx,new_idx,new_col,bar_width,width,height,title,xlab,ylab,legend_loc,legend_font,legend_ncol,txt_adjust_y,txt_lim,font,legend_alpha):
    cols = pd.Series(data.columns)[pd.Series(data.columns).str.contains(match_str,regex=False)].copy()

    df = data.loc[:,cols].copy()
    #df = df.iloc[:,:-2].copy()
    df = df.loc[:,list(pd.Series(df.columns).str.contains(" None",regex=False)==False)].copy()


    df[count_col] = df.apply(lambda x: len(pd.Series(x).dropna()),axis=1)
    df = pd.concat([data[idx],df[count_col]],axis=1).copy()

    df.columns = [new_idx,new_col]


    data_dist = pd.crosstab(index=df[new_idx],
                            columns=df[new_col])
    
    role_cnt = data[idx].value_counts()
    data_dist=data_dist.merge(role_cnt,how="left",left_index=True,right_index=True).copy()
    for i in data_dist.columns:
        data_dist.loc[:,i] = data_dist.loc[:,i] / data_dist.iloc[:,-1]
    
    data_dist.drop(columns=idx,inplace=True)
    
    
    data_dist1 = data_dist.copy()
    data_dist["sum_total"] = data_dist.sum(axis=1)
    
    for i in data_dist.columns:
        data_dist.loc[:,i] = data_dist.loc[:,i] / data_dist.iloc[:,-1]
    
    data_dist.drop(columns="sum_total",inplace=True)
    
    

    data_dist.plot(kind="barh",stacked=True,width=bar_width,figsize=(width,height),title=title,colormap="tab20")
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.xticks([])
    plt.legend(loc=legend_loc,fontsize=legend_font,ncol=legend_ncol,framealpha=legend_alpha)

    for n,i in enumerate(data_dist.index):
        l = np.cumsum(data_dist.loc[i,:])
        l = pd.Series([0]).append(l[:-1])
        l = np.array(l).copy()
        for k,j in enumerate(data_dist.columns):
            if np.round(data_dist1.loc[i,j]*100,1)>txt_lim:
                plt.text(x=l[k],y=n-txt_adjust_y,s=str(np.round(data_dist1.loc[i,j]*100,1))+"%",fontsize=font)
    facts = ""
    for indx in list(data_dist1.index):
        dat_row = data_dist1.loc[indx,:].sort_values(ascending=False).copy()
        if (indx == "Data Scientist") | (indx == "Machine Learning Engineer"):
            for colum in list(dat_row.index):
                if colum not in excl:
                    
                    facts+= "'" + " ".join((str(np.round(dat_row.loc[colum]*100,2)) + "%' of Respondants Who is/was a " + str(indx) + str(fill_txt) + "'" + str(colum)+"'" + str(fill_txt2)).split()) +"\n"
            facts+="\n\n"
    
    plt.show()
    print(facts)


    
    
def bivariate_dist_6(fill_txt,fill_txt2,excl,match_str,lang,idx,new_idx,new_col,bar_width,width,height,xlab,ylab,title,legend_loc,legend_font,legend_ncol,txt_adjust_y,txt_lim,font,legend_alpha):
    
    
    cols = pd.Series(data.columns)[pd.Series(data.columns).str.contains(match_str,regex=False)].copy()
    
    
    
    
    df = data.loc[:,cols].copy()
    df.fillna("",inplace=True)
    df[new_col] = df.apply(lambda x: ";".join(pd.Series(x).str.strip().sort_values()),axis=1)
    while any(df[new_col].str.contains(";;",regex=False)):
        df[new_col] = df[new_col].str.replace(";;",";",regex=False)
    df[new_col] = df[new_col].str.replace(";$"," ",regex=True)
    df[new_col] = df[new_col].str.replace("^;"," ",regex=True)
    df[new_col] = df[new_col].apply(lambda x: " ".join(x.strip().split()))
    df = df.loc[(df[new_col]!="") & (df[new_col]!=" ") & (df[new_col]!=";") & (df[new_col]!="; ") & (df[new_col]!=" ;"),:].copy()
    df = pd.concat([data[idx],df[new_col]],axis=1).copy()
    df.columns = [new_idx,new_col]
    


    
    data_dist = pd.crosstab(index=df[new_idx],
                            columns=df[new_col])
    
    role_cnt = data[idx].value_counts()
    data_dist=data_dist.merge(role_cnt,how="left",left_index=True,right_index=True).copy()
    for i in data_dist.columns:
        data_dist.loc[:,i] = data_dist.loc[:,i] / data_dist.iloc[:,-1]

    data_dist.drop(columns=idx,inplace=True)
    
   
    
    
    comb_df = pd.DataFrame()
    for i in range(len(lang)-1):
        for j in range(i+1, len(lang)):
            kdf = pd.DataFrame(data_dist.loc[:,list(((pd.Series(pd.Series(data_dist.columns)+";").str.contains(lang[i].strip()+";",regex=False)) & \
                                                     (pd.Series(pd.Series(data_dist.columns)+";").str.contains(lang[j].strip()+";",regex=False))))].sum(axis=1)).copy()
            kdf.columns = [";".join([lang[i].strip(),lang[j].strip()])]

            if (i == 0) & (j == 1):
                comb_df = kdf
            else:
                comb_df = comb_df.merge(kdf,how="left",left_index=True,right_index=True).copy()
    
    if len(lang)>=3:
        for i in range(len(lang)-2):
            for j in range(i+1, (len(lang)-1)):
                for k in range(j+1, len(lang)):
                    kdf = pd.DataFrame(data_dist.loc[:,list(((pd.Series(pd.Series(data_dist.columns)+";").str.contains(lang[i].strip()+";",regex=False)) & \
                                                             (pd.Series(pd.Series(data_dist.columns)+";").str.contains(lang[j].strip()+";",regex=False)) & \
                                                             (pd.Series(pd.Series(data_dist.columns)+";").str.contains(lang[k].strip()+";",regex=False))))].sum(axis=1)).copy()
                    kdf.columns = [";".join([lang[i].strip(),lang[j].strip(),lang[k].strip()])]
                    comb_df = comb_df.merge(kdf,how="left",left_index=True,right_index=True).copy()
        
    if len(lang)==4:
        kdf = pd.DataFrame(data_dist.loc[:,list(((pd.Series(pd.Series(data_dist.columns)+";").str.contains(lang[0].strip()+";",regex=False)) & \
                                                             (pd.Series(pd.Series(data_dist.columns)+";").str.contains(lang[1].strip()+";",regex=False)) & \
                                                             (pd.Series(pd.Series(data_dist.columns)+";").str.contains(lang[2].strip()+";",regex=False)) & \
                                                             (pd.Series(pd.Series(data_dist.columns)+";").str.contains(lang[3].strip()+";",regex=False))))].sum(axis=1)).copy()
        kdf.columns = [";".join([lang[0].strip(),lang[1].strip(),lang[2].strip(), lang[3].strip()])]
        comb_df = comb_df.merge(kdf,how="left",left_index=True,right_index=True).copy()
        

    
    
    data_dist = comb_df.copy()
    
    data_dist1 = data_dist.copy()
    data_dist["sum_total"] = data_dist.sum(axis=1)
    
    for i in data_dist.columns:
        data_dist.loc[:,i] = data_dist.loc[:,i] / data_dist.iloc[:,-1]
    
    data_dist.drop(columns="sum_total",inplace=True)
    
   

    
    data_dist.plot(kind="barh",stacked=True,width=bar_width,figsize=(width,height),title=title,colormap="tab20");
    plt.legend(loc=legend_loc,fontsize=legend_font,ncol=legend_ncol,framealpha=legend_alpha)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.xticks([])
    

    for n,i in enumerate(data_dist.index):
        l = np.cumsum(data_dist.loc[i,:])
        l = pd.Series([0]).append(l[:-1])
        l = np.array(l).copy()
        for k,j in enumerate(data_dist.columns):
            if np.round(data_dist1.loc[i,j]*100,1)>txt_lim:
                plt.text(x=l[k],y=n-txt_adjust_y,s=str(np.round(data_dist1.loc[i,j]*100,1))+"%",fontsize=font)
    facts = ""
    for indx in list(data_dist1.index):
        dat_row = data_dist1.loc[indx,:].sort_values(ascending=False).copy()
        if (indx == "Data Scientist") | (indx == "Machine Learning Engineer"):
            for colum in list(dat_row.index):
                if colum not in excl:
                    
                    facts+= "'" + " ".join((str(np.round(dat_row.loc[colum]*100,2)) + "%' of Respondants Who is/was a " + str(indx) + str(fill_txt) + "'" + str(colum)+"'" + str(fill_txt2)).split()) +"\n"
            facts+="\n\n"
    plt.figtext(x=0.1,y=0.1,s="*Proportions will not add up to 100%, since combinations of only a subset of responses are visualized.")
    plt.show()
    print(facts)
    


<a id="dist"></a><h1>Frequency Distribution of Roles</h1>

In [ ]:
univariate_dist(col='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',sort_index=False,title="Role Distribution",width=15,height=7,xlab="Role",ylab="Frequency (log scale)",txt_adjust_x=0.3,txt_adjust_y=2.8,logx=False,logy=True,txt_rot=0,txt_font=11)

<a id="age"></a><h1>Age Groups of Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist(fill_txt = " Have the Age in ",fill_txt2=" Years Range.",excl=[""],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',col='What is your age (# years)?',norm="index",bar_width=0.95,width=15,height=12,title="Role-wise Proportion of Age Groups",xlab="",ylab="Role",legend_loc="upper left",legend_font="small",legend_ncol=13,txt_font=10,txt_lim=2,legend_alpha=1)

<a id="edu"></a><h1>Education of Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist(fill_txt = " Have ",fill_txt2=".",excl=["I prefer not to answer"],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',col='What is the highest level of formal education that you have attained or plan to attain within the next 2 years?',norm="index",bar_width=0.95,width=15,height=12,title="Role-wise Proportion of Education",xlab="",ylab="Role",legend_loc="upper left",legend_font="small",legend_ncol=13,txt_font=10,txt_lim=2,legend_alpha=1)

<a id="gen"></a><h1>Gender of Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist(fill_txt = " is a ",fill_txt2=".",excl=["Prefer to self-describe","Prefer not to say"],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',col='What is your gender? - Selected Choice',norm="index",bar_width=0.95,width=15,height=12,title="Role-wise Proportion of Gender",xlab="",ylab="Role",legend_loc="upper left",legend_font="small",legend_ncol=13,txt_font=10,txt_lim=2,legend_alpha=1)

<a id="cexp"></a><h1>Coding Experience of Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist(fill_txt=" Have Coding/Programming Experience of ",fill_txt2=".",excl=["I have never written code"],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',col='For how many years have you been writing code and/or programming?',norm="index",bar_width=0.95,width=15,height=12,title="Role-wise Proportion of Coding Experience",xlab="",ylab="Role",legend_loc="upper left",legend_font="small",legend_ncol=13,txt_font=9.8,txt_lim=2.1,legend_alpha=1)

<a id="npl"></a><h1>Number of Programming Languages Used By Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist_3(fill_txt = " Use ",fill_txt2=" Programming language(s).",excl=[""],match_str='What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice -',count_col="Languages Used Count",idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Role",new_col="No. of Languages Used",bar_width=0.95,width=15,height=12,title="Role-wise Proportion of Number of Languages Used",xlab="",ylab="Role",legend_loc="upper left",legend_font="small",legend_ncol=13,txt_adjust_y=0.3,txt_lim=2,font=10,legend_alpha=1)

<a id="pl"></a><h1>Programming Languages Used By Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist_2(fill_txt = " Use ",fill_txt2=" (Along With Other Programming Languages).",excl=["None","Other"],match_str='What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice -',idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Languages Used",bar_width=0.95,width=15,height=12,xlab="",ylab="Role",title="Role-wise Proportion of Language Used",legend_loc="upper left",legend_font="small",legend_ncol=13,txt_adjust_y=0.3,txt_lim=5,font=8,legend_alpha=1)

<a id="mpl"></a><h1>Multiple Programming Languages Used By Data Scientist & ML Engineer</h1>

<h4>Analyzing the combinations of top 4 programming languages used by Data Scientists (Python, SQL, R, C++). Analyzing all the combinations makes the visualization cluttered</h4>

In [ ]:
bivariate_dist_6(fill_txt = " Use ",fill_txt2=" Along With Other Programming Languages.",excl=["None","Other"],match_str='What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice -',lang=["Python","SQL","R","C++"],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Languages Used",bar_width=0.95,width=15,height=12,xlab="",ylab="Role",title="Role-wise Proportion of Languages Used (Combination of Python; SQL; R; C++)",legend_loc="upper left",legend_font="small",legend_ncol=9,txt_adjust_y=0.3,txt_lim=4.4,font=10,legend_alpha=1)

In [ ]:
bivariate_dist_5(fill_txt = " Use Only ",fill_txt2=".",excl=["None","Other"],match_str='What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice -',lang=["Python","SQL","R","C++"],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Languages Used",bar_width=0.95,width=15,height=12,xlab="",ylab="Role",title="Role-wise Proportion of Languages Used (Combination of Python; SQL; R; C++)",legend_loc="upper left",legend_font="small",legend_ncol=9,txt_adjust_y=0.3,txt_lim=1,font=10,legend_alpha=1)

<a id="nide"></a><h1>Number of IDEs Used By Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist_3(fill_txt = " Use ",fill_txt2=" IDE(s).",excl=[""],match_str="Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -",count_col="IDEs Used Count",idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Role",new_col="No. of IDEs Used",bar_width=0.95,width=15,height=10,title="Role-wise Proportion of No. of IDEs Used",xlab="",ylab="Role",legend_loc="upper left",legend_font="small",legend_ncol=13,txt_adjust_y=0.3,txt_lim=1,font=9.7,legend_alpha=1)

<a id="ide"></a><h1>IDEs Used By Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist_2(fill_txt = " Use ",fill_txt2=" (Along With Other IDEs).",excl=["None","Other"],match_str="Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -",idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="IDEs Used",bar_width=0.95,width=15,height=12,xlab="",ylab="Current Role",title="Role-wise Proportion of IDE Used",legend_loc="upper left",legend_font="small",legend_ncol=6,txt_adjust_y=0.3,txt_lim=5.5,font=9.6,legend_alpha=1)

<a id="mide"></a><h1>Multiple IDEs Used By Data Scientist & ML Engineer</h1>

<h4>Analyzing the combinations of 4 IDEs used by Data Scientists (Jupyter, Spyder, Pycharm, RStudio). Analyzing all the combinations makes the visualization cluttered</h4>

In [ ]:
bivariate_dist_6(fill_txt = " Use ",fill_txt2=" Along With Other IDEs.",excl=["None","Other"],match_str="Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -",lang=['Jupyter (JupyterLab, Jupyter Notebooks, etc)',' PyCharm',' RStudio',' Spyder'],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Languages Used",bar_width=0.95,width=15,height=17,xlab="",ylab="Role",title="Role-wise Proportion of IDEs Used (Combination of Jupyter; Spyder; PyCharm; RStudio)",legend_loc="upper left",legend_font="small",legend_ncol=3,txt_adjust_y=0.45,txt_lim=2.5,font=10,legend_alpha=1)

In [ ]:
bivariate_dist_5(fill_txt = " Use Only ",fill_txt2=".",excl=["None","Other"],match_str="Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -",lang=['Jupyter (JupyterLab, Jupyter Notebooks, etc)',' PyCharm',' RStudio','  Spyder'],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Languages Used",bar_width=0.95,width=15,height=19,xlab="",ylab="Role",title="Role-wise Proportion of IDEs Used (Combination of Jupyter; Spyder; PyCharm; RStudio)",legend_loc="upper left",legend_font="small",legend_ncol=3,txt_adjust_y=0.45,txt_lim=1.5,font=10,legend_alpha=1)

<a id="nhnb"></a><h1>Number of Hosted Notebook Products Used By Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist_3(fill_txt = " Use ",fill_txt2=" Hosted Notebook Product(s).",excl=[""],match_str='Which of the following hosted notebook products do you use on a regular basis?  (Select all that apply) - Selected Choice -',count_col="hosted notebook products Used Count",idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Role",new_col="No. of hosted notebook products Used",bar_width=0.95,width=15,height=12,title="Role-wise Proportion of No. of Hosted Notebook Products Used",xlab="",ylab="Role",legend_loc="upper left",legend_font="small",legend_ncol=13,txt_adjust_y=0.1,txt_lim=2,font=10,legend_alpha=1)

<a id="hnb"></a><h1>Hosted Notebook Products Used By Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist_2(fill_txt = " Use ",fill_txt2=" (Along With Other Hosted Notebook Products).",excl=["None","Other"],match_str='Which of the following hosted notebook products do you use on a regular basis?  (Select all that apply) - Selected Choice -',idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Hosted Notebooks Used",bar_width=0.98,width=15,height=14,xlab="",ylab="Role",title="Role-wise Proportion of Hosted Notebook Product Used",legend_loc="upper left",legend_font="small",legend_ncol=6,txt_adjust_y=0.3,txt_lim=3.5,font=9.7,legend_alpha=1)

<a id="mhnb"></a><h1>Multiple Hosted Notebook Products Used By Data Scientist & ML Engineer</h1>

<h4>Analyzing the combinations of top 3 Hosted Notebook Products used by Data Scientists (Colab Notebooks, Kaggle Notebooks, Binder / JupyterHub). Analyzing all the combinations makes the visualization cluttered</h4>

In [ ]:
bivariate_dist_6(fill_txt = " Use ",fill_txt2=" Along With Other Hosted Notebook Products.",excl=[""],match_str='Which of the following hosted notebook products do you use on a regular basis?  (Select all that apply) - Selected Choice -',lang=['Colab Notebooks',' Kaggle Notebooks',' Binder / JupyterHub'],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Languages Used",bar_width=0.95,width=15,height=13,xlab="",ylab="Role",title="Role-wise Proportion of Hosted Notebook Products Used (Combinations of Colab Notebooks; Kaggle Notebooks; Binder / JupyterHub)",legend_loc="upper left",legend_font="small",legend_ncol=4,txt_adjust_y=0.3,txt_lim=1.2,font=10,legend_alpha=1)

In [ ]:
bivariate_dist_5(fill_txt = " Use Only ",fill_txt2=".",excl=[""],match_str='Which of the following hosted notebook products do you use on a regular basis?  (Select all that apply) - Selected Choice -',lang=['Colab Notebooks',' Kaggle Notebooks',' Binder / JupyterHub'],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Languages Used",bar_width=0.95,width=15,height=13,xlab="",ylab="Role",title="Role-wise Proportion of Hosted Notebook Products Used (Combinations of Colab Notebooks; Kaggle Notebooks; Binder / JupyterHub)",legend_loc="upper left",legend_font="small",legend_ncol=4,txt_adjust_y=0.3,txt_lim=1.2,font=10,legend_alpha=1)

<a id="cpds"></a><h1>Computing Platforms Used By Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist(fill_txt = " Use ",fill_txt2=" for their projects.",excl=["None","Other"],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',col='What type of computing platform do you use most often for your data science projects? - Selected Choice',norm="index",bar_width=0.95,width=15,height=13,title="Role-wise Proportion of Computing Platforms Used",xlab="",ylab="Role",legend_loc="upper left",legend_font="small",legend_ncol=3,txt_font=10,txt_lim=2.6,legend_alpha=1)

<a id="aspir"></a><h1>Recommended Language to Learn First By Data Science Aspirants</h1>

In [ ]:
bivariate_dist(fill_txt = " recommend an aspiring data scientist to learn ",fill_txt2=" first.",excl=["None","Other"],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',col='What programming language would you recommend an aspiring data scientist to learn first? - Selected Choice',norm="index",bar_width=0.95,width=15,height=12,title="Role-wise Proportion of Recommended Language To Learn",xlab="",ylab="Role",legend_loc="upper left",legend_font="small",legend_ncol=13,txt_font=9,txt_lim=2.3,legend_alpha=1)

<a id="shw"></a><h1>Specialized Harware Used By Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist_2(fill_txt = " use ",fill_txt2=" (specialized hardware).",excl=["Other","None"],match_str='Which types of specialized hardware do you use on a regular basis?  (Select all that apply) - Selected Choice -',idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Role",new_col="Specialized Hardware Used",bar_width=0.95,width=15,height=13,xlab="",ylab="Role",title="Role-wise Proportion of Specialized Hardware Used",legend_loc="upper left",legend_font="small",legend_ncol=7,txt_adjust_y=0.1,font=10,txt_lim=1.6,legend_alpha=1)

<a id="ndvl"></a><h1>Number of Data Visualization Libraries Used By Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist_3(fill_txt = " Use ",fill_txt2=" Data Visualization Libraries.",excl=["Other"],match_str='What data visualization libraries or tools do you use on a regular basis?  (Select all that apply) - Selected Choice -',count_col="Data Visualization Libraries Used Count",idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Role",new_col="No. of Data Visualization Libraries Used",bar_width=0.95,width=15,height=12,title="Role-wise Proportion of No. of Data Visualization Libraries Used",xlab="",ylab="Role",legend_loc="upper left",legend_font="small",legend_ncol=13,txt_adjust_y=0.1,font=10,txt_lim=1,legend_alpha=1)

<a id="dvlu"></a><h1>Data Visualization Libraries Used By Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist_2(fill_txt = " Use ",fill_txt2=" Along With Other Data Visualization Algorithms.",excl=["None","Other"],match_str='What data visualization libraries or tools do you use on a regular basis?  (Select all that apply) - Selected Choice -',idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Data Visualization Libraries Used",bar_width=0.96,width=15,height=12,xlab="",ylab="Role",title="Role-wise Proportion of Data Visualization Libraries Used",legend_loc="upper left",legend_font="small",legend_ncol=6,txt_adjust_y=0.3,font=9.5,txt_lim=5,legend_alpha=1)

<a id="muldvl"></a><h1>Multiple Data Visualization Libraries Used By Data Scientist & ML Engineer</h1>

<h4>Analyzing the combinations of top 4 Data Visualization Libraries used by Data Scientists (Matplotlib, Seaborn, Plotly, ggplot). Analyzing all the combinations makes the visualization cluttered</h4>

In [ ]:
bivariate_dist_6(fill_txt = " Use ",fill_txt2=" Along With Other Data Visualization Libraries.",excl=["None","Other"],match_str='What data visualization libraries or tools do you use on a regular basis?  (Select all that apply) - Selected Choice -',lang=[' Matplotlib',' Seaborn',' Plotly / Plotly Express',' Ggplot / ggplot2'],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Data Visualization Libraries Used",bar_width=0.98,width=15,height=15,xlab="",ylab="Role",title="Role-wise Proportion of Data Visualization Libraries Used (Combinations of Matplotlib;Seaborn;Plotly / Plotly Express; Ggplot / ggplot2)",legend_loc="upper left",legend_font="small",legend_ncol=4,txt_adjust_y=0.4,font=9.7,txt_lim=1.5,legend_alpha=1)

In [ ]:
bivariate_dist_5(fill_txt = " Use Only ",fill_txt2=".",excl=["None","Other"],match_str='What data visualization libraries or tools do you use on a regular basis?  (Select all that apply) - Selected Choice -',lang=[' Matplotlib',' Seaborn',' Plotly / Plotly Express',' Ggplot / ggplot2'],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Data Visualization Libraries Used",bar_width=0.98,width=15,height=15,xlab="",ylab="Role",title="Role-wise Proportion of Data Visualization Libraries Used (Combinations of Matplotlib;Seaborn;Plotly / Plotly Express; Ggplot / ggplot2)",legend_loc="upper left",legend_font="small",legend_ncol=4,txt_adjust_y=0.4,font=9.7,txt_lim=1.5,legend_alpha=1)

<a id="mlexp"></a><h1>Years of ML Experience of Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist(fill_txt = " has been using machine learning methods for ",fill_txt2=".",excl=["I do not use machine learning methods"],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',col='For how many years have you used machine learning methods?',norm="index",bar_width=0.95,width=15,height=12,title="Role-wise Proportion of Years of ML Experience",xlab="",ylab="Role",legend_loc="upper left",legend_font="small",legend_ncol=13,txt_font=10,txt_lim=2.1,legend_alpha=1)

<a id="nmlfw"></a><h1>Number of Machine Learning Frameworks Used By Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist_3(fill_txt = " Use ",fill_txt2=" Machine Learning Framework(s).",excl=["Other"],match_str='Which of the following machine learning frameworks do you use on a regular basis? (Select all that apply) - Selected Choice -',count_col="machine learning frameworks Used Count",idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Role",new_col="No. of machine learning frameworks Used",bar_width=0.95,width=15,height=12,title="Role-wise Proportion of No. of Machine Learning Frameworks Used",xlab="",ylab="Role",legend_loc="upper left",legend_font="small",legend_ncol=14,txt_adjust_y=0.1,font=10,txt_lim=1.5,legend_alpha=1)

<a id="mlfwu"></a><h1>Machine Learning Frameworks Used By Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist_2(fill_txt = " Use ",fill_txt2=" Along With Other Machine Learning Frameworks.",excl=["None","Other"],match_str='Which of the following machine learning frameworks do you use on a regular basis? (Select all that apply) - Selected Choice -',idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Machine Learning Frameworks Used",bar_width=0.98,width=15,height=14,xlab="",ylab="Role",title="Role-wise Proportion of Machine Learning Frameworks Used",legend_loc="upper left",legend_font="small",legend_ncol=8,txt_adjust_y=0.1,font=9.5,txt_lim=7,legend_alpha=1)

<a id="mulmlfw"></a><h1>Multiple Machine Learning Frameworks Used By Data Scientist & ML Engineer</h1>

<h4>Analyzing the combinations of top 4 Machine Learning Frameworks used by Data Scientists (Scikit-Learn, Keras, TensorFlow, XgBoost). Analyzing all the combinations makes the visualization cluttered</h4>

In [ ]:
bivariate_dist_6(fill_txt = " Use ",fill_txt2=" Along With Other Machine Learning Frameworks.",excl=["None","Other"],match_str='Which of the following machine learning frameworks do you use on a regular basis? (Select all that apply) - Selected Choice -',lang=['  Scikit-learn',' Keras','  TensorFlow',' Xgboost'],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Data Visualization Libraries Used",bar_width=0.95,width=15,height=14,xlab="",ylab="Role",title="Role-wise Proportion of Machine Learning Frameworks Used (Combinations of Scikit-Learn; Keras; TensorFlow; Xgboost)",legend_loc="upper left",legend_font="small",legend_ncol=5,txt_adjust_y=0.35,font=10,txt_lim=1.5,legend_alpha=1)

In [ ]:
bivariate_dist_5(fill_txt = " Use Only ",fill_txt2=".",excl=["None","Other"],match_str='Which of the following machine learning frameworks do you use on a regular basis? (Select all that apply) - Selected Choice -',lang=['  Scikit-learn',' Keras','  TensorFlow',' Xgboost'],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Data Visualization Libraries Used",bar_width=0.95,width=15,height=14,xlab="",ylab="Role",title="Role-wise Proportion of Machine Learning Frameworks Used (Combinations of Scikit-Learn; Keras; TensorFlow; Xgboost)",legend_loc="upper left",legend_font="small",legend_ncol=5,txt_adjust_y=0.35,font=10,txt_lim=1.5,legend_alpha=1)

<a id="nmlalg"></a><h1>Number of Machine Learning Algorithms Used By Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist_3(fill_txt = " Use ",fill_txt2=" Machine Learning Algorithm(s).",excl=["Other"],match_str='Which of the following ML algorithms do you use on a regular basis? (Select all that apply): - Selected Choice -',count_col="machine learning frameworks Used Count",idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Role",new_col="No. of machine learning frameworks Used",bar_width=0.95,width=15,height=12,title="Role-wise Proportion of No. of Machine Learning Algorithms Used",xlab="",ylab="Role",legend_loc="upper left",legend_font="small",legend_ncol=14,txt_adjust_y=0.1,font=10,txt_lim=1.5,legend_alpha=1)

<a id="mlalg"></a><h1>Machine Learning Algorithms Used By Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist_2(fill_txt = " Use ",fill_txt2=" Along With Other Machine Learning Algorithms.",excl=["None","Other"],match_str='Which of the following ML algorithms do you use on a regular basis? (Select all that apply): - Selected Choice -',idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Machine Learning Frameworks Used",bar_width=0.98,width=15,height=14,xlab="",ylab="Role",title="Role-wise Proportion of Machine Learning Algorithms Used",legend_loc="upper left",legend_font="small",legend_ncol=4,txt_adjust_y=0.3,font=9.5,txt_lim=7,legend_alpha=1)

<a id="mulmlalg"></a><h1>Multiple Machine Learning Algorithms Used By Data Scientist & ML Engineer</h1>

<h4>Analyzing the combinations of top 3 Machine Learning Algorithms used by Data Scientists (Linear or Logistic Regression, Decision Trees or Random Forests, and Gradient Boosting Machines). Analyzing all the combinations makes the visualization cluttered</h4>

In [ ]:
bivariate_dist_6(fill_txt = " Use ",fill_txt2=" Along With Other Machine Learning Algorithms.",excl=["None","Other"],match_str='Which of the following ML algorithms do you use on a regular basis? (Select all that apply): - Selected Choice -',lang=['Linear or Logistic Regression', 'Decision Trees or Random Forests', 'Gradient Boosting Machines (xgboost, lightgbm, etc)'],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Data Visualization Libraries Used",bar_width=0.95,width=15,height=16,xlab="",ylab="Role",title="Role-wise Proportion of Machine Learning Algorithms Used (Combinations of Linear or Logistic Regression; Decision Trees or Random Forests; Gradient Boosting Machines)",legend_loc="upper left",legend_font="small",legend_ncol=2,txt_adjust_y=0.35,font=10,txt_lim=1.5,legend_alpha=1)

In [ ]:
bivariate_dist_5(fill_txt = " Use Only ",fill_txt2=".",excl=["None","Other"],match_str='Which of the following ML algorithms do you use on a regular basis? (Select all that apply): - Selected Choice -',lang=['Linear or Logistic Regression', 'Decision Trees or Random Forests', 'Gradient Boosting Machines (xgboost, lightgbm, etc)'],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Data Visualization Libraries Used",bar_width=0.95,width=15,height=16,xlab="",ylab="Role",title="Role-wise Proportion of Machine Learning Algorithms Used (Combinations of Linear or Logistic Regression; Decision Trees or Random Forests; Gradient Boosting Machines)",legend_loc="upper left",legend_font="small",legend_ncol=2,txt_adjust_y=0.35,font=10,txt_lim=1.5,legend_alpha=1)

<a id="ncvalg"></a><h1>Number of Computer Vision Methods Used By Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist_3(fill_txt = " Use ",fill_txt2=" Computer Vision Method(s).",excl=[],match_str='Which categories of computer vision methods do you use on a regular basis?  (Select all that apply) - Selected Choice -',count_col="machine learning frameworks Used Count",idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Role",new_col="No. of machine learning frameworks Used",bar_width=0.95,width=15,height=12,title="Role-wise Proportion of No. of Computer Vision Methods Used",xlab="",ylab="Role",legend_loc="upper left",legend_font="small",legend_ncol=14,txt_adjust_y=0.1,font=10,txt_lim=1.5,legend_alpha=1)

<a id="cvalg"></a><h1>Computer Vision Methods Used By Data Scientist & ML Engineer</h1>

In [ ]:
bivariate_dist_2(fill_txt = " Use ",fill_txt2=" Along With Other Computer Vision Methods.",excl=["None","Other"],match_str='Which categories of computer vision methods do you use on a regular basis?  (Select all that apply) - Selected Choice -',idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Machine Learning Frameworks Used",bar_width=0.98,width=15,height=18,xlab="",ylab="Role",title="Role-wise Proportion of Computer Vision Methods Used",legend_loc="upper left",legend_font="small",legend_ncol=2,txt_adjust_y=0.3,font=9.5,txt_lim=0.5,legend_alpha=1)

<a id="mulcvalg"></a><h1>Multiple Computer Vision Methods Used By Data Scientist & ML Engineer</h1>

<h4>Analyzing the combinations of top 3 Computer Vision Methods used by Data Scientists (Image classification and other general purpose networks; General purpose image/video tools; and Image segmentation methods). Analyzing all the combinations makes the visualization cluttered</h4>

In [ ]:
bivariate_dist_6(fill_txt = " Use ",fill_txt2=" Along With Other Computer Vision Methods.",excl=["None","Other"],match_str='Which categories of computer vision methods do you use on a regular basis?  (Select all that apply) - Selected Choice -',lang=['Image classification and other general purpose networks (VGG, Inception, ResNet, ResNeXt, NASNet, EfficientNet, etc)', 'General purpose image/video tools (PIL, cv2, skimage, etc)', 'Image segmentation methods (U-Net, Mask R-CNN, etc)'],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Data Visualization Libraries Used",bar_width=0.95,width=15,height=16,xlab="",ylab="Role",title="Role-wise Proportion of Computer Vision Methods Used (Combinations of Image classification and other general purpose networks; General purpose image/video tools; and Image segmentation methods)",legend_loc="upper left",legend_font="small",legend_ncol=1,txt_adjust_y=0.35,font=10,txt_lim=0,legend_alpha=1)

In [ ]:
bivariate_dist_5(fill_txt = " Use Only ",fill_txt2=".",excl=["None","Other"],match_str='Which categories of computer vision methods do you use on a regular basis?  (Select all that apply) - Selected Choice -',lang=['Image classification and other general purpose networks (VGG, Inception, ResNet, ResNeXt, NASNet, EfficientNet, etc)', 'General purpose image/video tools (PIL, cv2, skimage, etc)', 'Image segmentation methods (U-Net, Mask R-CNN, etc)'],idx='Select the title most similar to your current role (or most recent title if retired): - Selected Choice',new_idx="Current Role",new_col="Data Visualization Libraries Used",bar_width=0.95,width=15,height=16,xlab="",ylab="Role",title="Role-wise Proportion of Computer Vision Methods Used (Combinations of Image classification and other general purpose networks; General purpose image/video tools; and Image segmentation methods)",legend_loc="upper left",legend_font="small",legend_ncol=1,txt_adjust_y=0.35,font=10,txt_lim=0.3,legend_alpha=0.5)